# 🟣 Concurrence et Asyncio

**Badge:** 🟣 Expert | ⏱ 60 min | 🔑 **Concepts clés :** threading, multiprocessing, GIL, asyncio

## Objectifs

- Comprendre la différence entre concurrence et parallélisme
- Maîtriser le GIL (Global Interpreter Lock) et son impact
- Utiliser `threading` pour les tâches I/O bound
- Utiliser `multiprocessing` pour les tâches CPU bound
- Découvrir la programmation asynchrone avec `asyncio`
- Choisir la bonne approche selon le cas d'usage

## Prérequis

- Compréhension des processus et threads
- Bases de Python (fonctions, décorateurs)
- Notion de latence réseau et I/O

## 1. Concurrence vs Parallélisme

### Définitions

**Concurrence** : Gérer plusieurs tâches en même temps (switching rapide)  
- Une seule tâche s'exécute à un instant T
- Utile quand les tâches attendent (I/O, réseau)
- Exemple : serveur web qui gère 1000 requêtes

**Parallélisme** : Exécuter plusieurs tâches simultanément (vraiment en parallèle)  
- Plusieurs tâches s'exécutent en même temps
- Nécessite plusieurs CPU cores
- Exemple : traiter 4 images en même temps sur 4 cores

```
Concurrence (1 CPU):
Task A: ████░░░░████░░░░
Task B: ░░░░████░░░░████

Parallélisme (2 CPUs):
Task A: ████████████████
Task B: ████████████████
```

## 2. Le GIL (Global Interpreter Lock)

### Qu'est-ce que le GIL ?

Le **GIL** est un verrou qui empêche plusieurs threads Python d'exécuter du bytecode Python en même temps.

**Conséquences** :
- ✅ Threading fonctionne bien pour I/O bound (attente réseau/disque)
- ❌ Threading ne parallélise PAS les calculs CPU
- ✅ Multiprocessing contourne le GIL (processus séparés)

**Note** : Python 3.13+ introduit une option "free-threaded" (pas de GIL), mais expérimental.

In [ ]:
import time
import threading

# Démonstration du GIL
def cpu_intensive():
    """Calcul intensif (CPU bound)"""
    count = 0
    for _ in range(10_000_000):
        count += 1
    return count

# Version séquentielle
start = time.time()
cpu_intensive()
cpu_intensive()
sequential_time = time.time() - start
print(f"Séquentiel : {sequential_time:.3f}s")

# Version threading (ne gagne rien à cause du GIL)
start = time.time()
t1 = threading.Thread(target=cpu_intensive)
t2 = threading.Thread(target=cpu_intensive)
t1.start()
t2.start()
t1.join()
t2.join()
threading_time = time.time() - start
print(f"Threading : {threading_time:.3f}s")

print(f"\n⚠️ Threading est {threading_time/sequential_time:.1f}x plus lent (à cause du GIL)")
print("Pour CPU bound, utilisez multiprocessing !")

## 3. Threading : Pour les tâches I/O bound

### Cas d'usage
- Téléchargements HTTP
- Lecture/écriture de fichiers
- Requêtes base de données
- Tout ce qui attend des ressources externes

In [ ]:
import threading
import time
import random

# Simuler un appel API lent
def fetch_data(api_id):
    """Simule un appel API avec latence réseau"""
    print(f"[Thread {threading.current_thread().name}] Fetch API {api_id}...")
    time.sleep(random.uniform(0.5, 2))  # Simulation latence réseau
    print(f"[Thread {threading.current_thread().name}] API {api_id} done")
    return f"Data from API {api_id}"

# Version séquentielle
print("=== VERSION SÉQUENTIELLE ===")
start = time.time()
results = []
for i in range(5):
    results.append(fetch_data(i))
seq_time = time.time() - start
print(f"Temps total : {seq_time:.2f}s\n")

# Version threading
print("=== VERSION THREADING ===")
start = time.time()
threads = []
results_threaded = []

for i in range(5):
    thread = threading.Thread(target=lambda x: results_threaded.append(fetch_data(x)), args=(i,))
    threads.append(thread)
    thread.start()

# Attendre que tous les threads se terminent
for thread in threads:
    thread.join()

thread_time = time.time() - start
print(f"Temps total : {thread_time:.2f}s")
print(f"\n✓ Gain : {seq_time/thread_time:.1f}x plus rapide avec threading")

### ThreadPoolExecutor : Interface moderne

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# Simuler le scraping de plusieurs URLs
def scrape_url(url):
    """Simule le scraping d'une URL"""
    time.sleep(random.uniform(0.3, 1))
    return {"url": url, "status": 200, "content_length": random.randint(1000, 50000)}

urls = [
    "https://api.example.com/products/1",
    "https://api.example.com/products/2",
    "https://api.example.com/products/3",
    "https://api.example.com/products/4",
    "https://api.example.com/products/5",
    "https://api.example.com/products/6",
]

print("Scraping avec ThreadPoolExecutor...\n")
start = time.time()

# max_workers : nombre de threads dans le pool
with ThreadPoolExecutor(max_workers=3) as executor:
    # Soumettre toutes les tâches
    futures = {executor.submit(scrape_url, url): url for url in urls}
    
    # Traiter les résultats au fur et à mesure
    for future in as_completed(futures):
        url = futures[future]
        try:
            result = future.result()
            print(f"✓ {url}: {result['content_length']} bytes")
        except Exception as e:
            print(f"❌ {url}: {e}")

print(f"\nTemps total : {time.time() - start:.2f}s")
print(f"Avec {len(urls)} URLs et 3 workers")

## 4. Multiprocessing : Pour les tâches CPU bound

### Cas d'usage
- Calculs mathématiques lourds
- Traitement d'images
- Machine learning (training)
- Parsing de gros fichiers

Multiprocessing crée de **vrais processus** séparés, chacun avec son propre GIL.

In [ ]:
import multiprocessing as mp
import os

def heavy_computation(n):
    """Calcul intensif : somme des carrés"""
    print(f"[Process {os.getpid()}] Calcul pour n={n}")
    result = sum(i*i for i in range(n))
    return result

# Version séquentielle
print("=== VERSION SÉQUENTIELLE ===")
numbers = [5_000_000, 6_000_000, 7_000_000, 8_000_000]

start = time.time()
results_seq = [heavy_computation(n) for n in numbers]
seq_time = time.time() - start
print(f"Temps : {seq_time:.2f}s\n")

# Version multiprocessing
print("=== VERSION MULTIPROCESSING ===")
start = time.time()

# Pool de processus
with mp.Pool(processes=4) as pool:
    results_mp = pool.map(heavy_computation, numbers)

mp_time = time.time() - start
print(f"Temps : {mp_time:.2f}s")
print(f"\n✓ Gain : {seq_time/mp_time:.1f}x plus rapide avec multiprocessing")
print(f"CPUs disponibles : {mp.cpu_count()}")

### ProcessPoolExecutor : Interface moderne

In [ ]:
from concurrent.futures import ProcessPoolExecutor
import numpy as np

def process_chunk(data_chunk):
    """Traite un chunk de données (simule un calcul lourd)"""
    # Calculs statistiques
    result = {
        'mean': np.mean(data_chunk),
        'std': np.std(data_chunk),
        'sum': np.sum(data_chunk),
        'size': len(data_chunk)
    }
    return result

# Créer un gros dataset
full_data = np.random.randn(10_000_000)

# Diviser en chunks
num_chunks = 4
chunks = np.array_split(full_data, num_chunks)

print(f"Dataset : {len(full_data):,} valeurs")
print(f"Chunks : {num_chunks} de ~{len(chunks[0]):,} valeurs\n")

# Traitement parallèle
start = time.time()
with ProcessPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(process_chunk, chunks))

print("Résultats par chunk :")
for i, res in enumerate(results):
    print(f"  Chunk {i}: mean={res['mean']:.4f}, std={res['std']:.4f}")

print(f"\nTemps : {time.time() - start:.2f}s")

## 5. Asyncio : Programmation asynchrone

### Concepts clés

- **async def** : définit une coroutine (fonction asynchrone)
- **await** : attend qu'une coroutine se termine
- **Event loop** : orchestre l'exécution des coroutines
- **asyncio.gather()** : exécute plusieurs coroutines en parallèle

**Avantages** :
- Très efficace pour I/O bound (milliers de connexions)
- Moins de overhead que threading
- Code plus lisible (pas de callbacks)

**Inconvénients** :
- Nécessite des bibliothèques async (aiohttp, asyncpg, etc.)
- Courbe d'apprentissage
- Ne résout PAS les problèmes CPU bound

In [ ]:
import asyncio

# Définir une coroutine
async def fetch_data_async(api_id):
    """Simule un appel API asynchrone"""
    print(f"[Coroutine] Fetch API {api_id}...")
    await asyncio.sleep(random.uniform(0.5, 1.5))  # await au lieu de time.sleep
    print(f"[Coroutine] API {api_id} done")
    return f"Data from API {api_id}"

# Exécuter plusieurs coroutines en parallèle
async def main():
    print("=== VERSION ASYNCIO ===")
    start = time.time()
    
    # Créer des tâches
    tasks = [fetch_data_async(i) for i in range(5)]
    
    # Exécuter en parallèle avec gather
    results = await asyncio.gather(*tasks)
    
    print(f"\nTemps total : {time.time() - start:.2f}s")
    print(f"Résultats : {len(results)} APIs fetched")
    return results

# Exécuter l'event loop
results = await main()  # Dans Jupyter, utilisez 'await' directement

### Asyncio avec aiohttp : Requêtes HTTP asynchrones

In [ ]:
# Installation : pip install aiohttp
# import aiohttp

# Exemple (nécessite aiohttp installé)
async def fetch_url_async(session, url):
    """Fetch une URL de manière asynchrone"""
    # async with session.get(url) as response:
    #     return await response.text()
    
    # Simulation sans aiohttp
    await asyncio.sleep(0.5)
    return f"Content from {url}"

async def download_all(urls):
    """Télécharge plusieurs URLs en parallèle"""
    # async with aiohttp.ClientSession() as session:
    #     tasks = [fetch_url_async(session, url) for url in urls]
    #     return await asyncio.gather(*tasks)
    
    # Simulation
    tasks = [fetch_url_async(None, url) for url in urls]
    return await asyncio.gather(*tasks)

# Test
urls = [
    "https://api.example.com/endpoint1",
    "https://api.example.com/endpoint2",
    "https://api.example.com/endpoint3",
]

start = time.time()
results = await download_all(urls)
print(f"✓ {len(results)} URLs téléchargées en {time.time() - start:.2f}s")

print("\n💡 Installer aiohttp pour de vraies requêtes HTTP async")

## 6. Comparaison : Quand utiliser quoi ?

| Type de tâche | Recommandation | Raison |
|---------------|----------------|--------|
| **I/O bound** (réseau, API, fichiers) | `asyncio` ou `threading` | Peu d'overhead, GIL n'est pas un problème |
| **CPU bound** (calculs, parsing) | `multiprocessing` | Contourne le GIL, utilise tous les cores |
| **I/O + CPU mixte** | `multiprocessing` + `asyncio` | Processus parallèles avec I/O async dans chacun |
| **Scraping web simple** | `threading` + `requests` | Simple, efficace |
| **Scraping web massif** | `asyncio` + `aiohttp` | Milliers de connexions simultanées |
| **Data processing** | `multiprocessing` + `pandas` | Traiter des chunks en parallèle |

### Règle simple

```python
if tache_attend_beaucoup:  # I/O, réseau
    use(asyncio)  # ou threading
elif tache_calcule_beaucoup:  # CPU
    use(multiprocessing)
```

## 7. Cas data engineering : Traitement parallèle de fichiers

In [ ]:
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path

# Créer plusieurs fichiers CSV à traiter
def create_sample_files(num_files=4):
    """Crée des fichiers CSV de test"""
    Path('data').mkdir(exist_ok=True)
    
    for i in range(num_files):
        df = pd.DataFrame({
            'order_id': range(i*1000, (i+1)*1000),
            'amount': np.random.uniform(10, 1000, 1000),
            'customer_id': np.random.randint(1, 100, 1000),
            'date': pd.date_range('2024-01-01', periods=1000, freq='H')
        })
        df.to_csv(f'data/orders_{i}.csv', index=False)
    
    print(f"✓ {num_files} fichiers créés dans ./data/")

def process_file(filepath):
    """Traite un fichier CSV (calculs statistiques)"""
    df = pd.read_csv(filepath)
    
    # Simule un traitement
    summary = {
        'file': filepath.name,
        'rows': len(df),
        'total_amount': df['amount'].sum(),
        'avg_amount': df['amount'].mean(),
        'unique_customers': df['customer_id'].nunique()
    }
    
    print(f"  Processed: {filepath.name}")
    return summary

# Créer les fichiers
create_sample_files(4)

# Traiter en parallèle
files = list(Path('data').glob('orders_*.csv'))
print(f"\nTraitement de {len(files)} fichiers en parallèle...")

start = time.time()
with ProcessPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(process_file, files))

# Consolider les résultats
df_summary = pd.DataFrame(results)
print(f"\n✓ Traitement terminé en {time.time() - start:.2f}s")
print("\nRésumé :")
print(df_summary)

print(f"\nTotal : {df_summary['total_amount'].sum():.2f}€")
print(f"Clients uniques : {df_summary['unique_customers'].sum()}")

## 8. Pattern avancé : Producteur-Consommateur avec Queue

In [ ]:
import queue
import threading

# Queue thread-safe pour communication entre threads
task_queue = queue.Queue()
result_queue = queue.Queue()

def producer(n_tasks):
    """Producteur : génère des tâches"""
    print(f"[Producer] Génération de {n_tasks} tâches")
    for i in range(n_tasks):
        task_queue.put(i)
        time.sleep(0.1)
    
    # Signal de fin
    task_queue.put(None)
    print("[Producer] Terminé")

def consumer(consumer_id):
    """Consommateur : traite les tâches"""
    while True:
        task = task_queue.get()
        
        if task is None:
            task_queue.put(None)  # Propager le signal aux autres consommateurs
            break
        
        # Traiter la tâche
        result = task * task
        result_queue.put((task, result))
        print(f"[Consumer {consumer_id}] Task {task} → {result}")
        time.sleep(0.2)
        
        task_queue.task_done()
    
    print(f"[Consumer {consumer_id}] Terminé")

# Lancer producteur et consommateurs
producer_thread = threading.Thread(target=producer, args=(10,))
consumer_threads = [
    threading.Thread(target=consumer, args=(i,))
    for i in range(3)
]

producer_thread.start()
for ct in consumer_threads:
    ct.start()

# Attendre la fin
producer_thread.join()
for ct in consumer_threads:
    ct.join()

# Récupérer les résultats
results = []
while not result_queue.empty():
    results.append(result_queue.get())

print(f"\n✓ {len(results)} résultats collectés")

## Pièges courants

### 1. Race conditions (accès concurrent à une variable)

In [ ]:
# ❌ Race condition
counter = 0

def increment_unsafe():
    global counter
    for _ in range(100000):
        counter += 1  # Non atomique !

threads = [threading.Thread(target=increment_unsafe) for _ in range(5)]
for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"Counter (unsafe) : {counter} (devrait être 500000)")

# ✅ Solution : Lock
counter_safe = 0
lock = threading.Lock()

def increment_safe():
    global counter_safe
    for _ in range(100000):
        with lock:  # Section critique
            counter_safe += 1

threads = [threading.Thread(target=increment_safe) for _ in range(5)]
for t in threads:
    t.start()
for t in threads:
    t.join()

print(f"Counter (safe) : {counter_safe} ✓")

### 2. Deadlock (interblocage)

In [ ]:
# ❌ Deadlock potentiel
lock1 = threading.Lock()
lock2 = threading.Lock()

def task_a():
    with lock1:
        time.sleep(0.1)
        # with lock2:  # Décommenter pour deadlock
        #     pass
        pass

def task_b():
    with lock2:
        time.sleep(0.1)
        # with lock1:  # Décommenter pour deadlock
        #     pass
        pass

# ✅ Solution : toujours acquérir les locks dans le même ordre
print("✓ Pour éviter deadlock : ordre fixe des locks")

### 3. Confondre async et threading

In [ ]:
# ❌ time.sleep() dans une coroutine async bloque l'event loop
async def bad_async():
    # time.sleep(1)  # ❌ Bloque TOUT
    await asyncio.sleep(1)  # ✅ Libère l'event loop

print("✓ Dans async : utilisez await asyncio.sleep(), pas time.sleep()")

### 4. Croire que async accélère le CPU

In [ ]:
# ❌ Async ne résout PAS les problèmes CPU bound
async def cpu_task():
    result = sum(i*i for i in range(10_000_000))
    return result

# Même avec asyncio.gather, c'est séquentiel (GIL)
# Pour CPU bound → multiprocessing !

print("✓ Async = I/O bound uniquement")
print("✓ CPU bound = multiprocessing")

## Mini-exercices

### Exercice 1 : Téléchargement parallèle

Simulez le téléchargement de 10 fichiers (sleep random 0.5-2s).  
Comparez le temps séquentiel vs threading vs asyncio.

In [ ]:
# Votre code ici


### Exercice 2 : Traitement CPU avec multiprocessing

Créez une fonction qui calcule la somme des carrés jusqu'à N.  
Traitez [1M, 2M, 3M, 4M, 5M] avec multiprocessing et comparez avec séquentiel.

In [ ]:
# Votre code ici


### Exercice 3 : Pipeline ETL parallèle

Créez 5 fichiers CSV avec des données aléatoires.  
Traitez-les en parallèle avec ProcessPoolExecutor :  
- Lisez chaque fichier  
- Appliquez une transformation (ex: multiplier une colonne par 2)  
- Sauvegardez dans un nouveau fichier  
Mesurez le temps.

In [ ]:
# Votre code ici


## Solutions des exercices

In [ ]:
# Solution Exercice 1
def download_file(file_id):
    time.sleep(random.uniform(0.5, 2))
    return f"file_{file_id}.dat"

async def download_file_async(file_id):
    await asyncio.sleep(random.uniform(0.5, 2))
    return f"file_{file_id}.dat"

file_ids = list(range(10))

# Séquentiel
print("=== SÉQUENTIEL ===")
start = time.time()
results_seq = [download_file(fid) for fid in file_ids]
seq_time = time.time() - start
print(f"Temps : {seq_time:.2f}s\n")

# Threading
print("=== THREADING ===")
start = time.time()
with ThreadPoolExecutor(max_workers=5) as executor:
    results_thread = list(executor.map(download_file, file_ids))
thread_time = time.time() - start
print(f"Temps : {thread_time:.2f}s\n")

# Asyncio
print("=== ASYNCIO ===")
start = time.time()
results_async = await asyncio.gather(*[download_file_async(fid) for fid in file_ids])
async_time = time.time() - start
print(f"Temps : {async_time:.2f}s\n")

print("COMPARAISON :")
print(f"  Séquentiel : {seq_time:.2f}s (baseline)")
print(f"  Threading  : {thread_time:.2f}s (gain {seq_time/thread_time:.1f}x)")
print(f"  Asyncio    : {async_time:.2f}s (gain {seq_time/async_time:.1f}x)")

In [ ]:
# Solution Exercice 2
def sum_of_squares(n):
    return sum(i*i for i in range(n))

numbers = [1_000_000, 2_000_000, 3_000_000, 4_000_000, 5_000_000]

# Séquentiel
print("=== SÉQUENTIEL ===")
start = time.time()
results_seq = [sum_of_squares(n) for n in numbers]
seq_time = time.time() - start
print(f"Temps : {seq_time:.2f}s\n")

# Multiprocessing
print("=== MULTIPROCESSING ===")
start = time.time()
with ProcessPoolExecutor(max_workers=5) as executor:
    results_mp = list(executor.map(sum_of_squares, numbers))
mp_time = time.time() - start
print(f"Temps : {mp_time:.2f}s\n")

print(f"✓ Gain : {seq_time/mp_time:.1f}x plus rapide avec multiprocessing")
print(f"CPU cores : {mp.cpu_count()}")

In [ ]:
# Solution Exercice 3
# 1. Créer fichiers
Path('input').mkdir(exist_ok=True)
Path('output').mkdir(exist_ok=True)

for i in range(5):
    df = pd.DataFrame({
        'id': range(i*100, (i+1)*100),
        'value': np.random.randn(100),
        'category': np.random.choice(['A', 'B', 'C'], 100)
    })
    df.to_csv(f'input/data_{i}.csv', index=False)

print("✓ 5 fichiers créés")

# 2. Fonction de traitement
def transform_file(input_path):
    df = pd.read_csv(input_path)
    
    # Transformation
    df['value'] = df['value'] * 2
    df['processed'] = True
    
    # Sauvegarder
    output_path = Path('output') / input_path.name
    df.to_csv(output_path, index=False)
    
    return output_path.name

# 3. Traiter en parallèle
input_files = list(Path('input').glob('data_*.csv'))

print(f"\nTraitement de {len(input_files)} fichiers...")
start = time.time()

with ProcessPoolExecutor(max_workers=3) as executor:
    output_files = list(executor.map(transform_file, input_files))

print(f"✓ Traitement terminé en {time.time() - start:.2f}s")
print(f"Fichiers générés : {output_files}")

## Résumé

### Points clés

1. **Concurrence** : gérer plusieurs tâches (switching) vs **Parallélisme** : exécuter simultanément
2. **GIL** : empêche le vrai parallélisme des threads Python pour CPU bound
3. **threading** : idéal pour I/O bound (réseau, fichiers, API)
4. **multiprocessing** : obligatoire pour CPU bound (calculs, traitement)
5. **asyncio** : très efficace pour I/O bound avec milliers de connexions
6. **ThreadPoolExecutor / ProcessPoolExecutor** : interfaces modernes et simples
7. **Race conditions** : utilisez des Lock pour protéger les ressources partagées

### Table de décision

| Tâche | Solution | Bibliothèque |
|-------|----------|-------------|
| Téléchargement API | asyncio + aiohttp | aiohttp |
| Scraping simple | threading + requests | requests |
| Calculs lourds | multiprocessing | concurrent.futures |
| Traitement fichiers | multiprocessing + pandas | pandas |
| Serveur web | asyncio | FastAPI, aiohttp |

### Prochaines étapes

- Notebook suivant : **Formats de données**
- Approfondir : asyncio patterns, distributed computing (Celery, Ray)